# Exercises for 2021-03-25

For this notebook, we will want to use the `MatrixNetworks` packages.  You can install it from the Julia command line (my preference) or execute a cell with the lines:

    using Pkg
    Pkg.add("MatrixNetworks")

In [ ]:
using LinearAlgebra
using Plots
using MatrixNetworks
using SparseArrays

## Classical Multi-Dimensional Scaling (MDS)

The goal of multi-dimensional scaling (MDS) is to recover a low-dimensional coordinate system that reflects pairwise distance data.  Consider the data matrix 
$X = \begin{bmatrix} x^1 & x^2 & \ldots & x^n \end{bmatrix} \in \mathbb{R}^{d \times n}$; the pairwise squared 
distances are
$$
  d_{ij} = \|x^i-x^j\|^2 = \|x^i\|^2 - 2 \langle x^i, x^j \rangle + \|x^j\|^2.
$$
In matrix form, we can write this as
$$
  D = \ell e^T - 2 X^T X + e \ell^T
$$
where $\ell_i = \|x_i\|^2$ and $e$ is the vector of all ones.  Now consider the centering transformation that maps a data vector to the vector minus the mean:
$$
  P = I - \frac{1}{n} ee^T;
$$
note that
$$
  B = -\frac{1}{2} PDP = PX^T XP = \bar{X}^T \bar{X}
$$
where $\bar{X} = XP$ is the centered data matrix (i.e. where we subtract the mean vector off each column).  The idea of MDS is to reconstruct $\bar{X}$ (or an equivalent coordinate system) via the eigenvectors of $B$.  A key point is that if the data clusters close to a subspace with fewer than $d$ dimensions, we can recover a coordinate system for that lower-dimensional subspace.

In [ ]:
# Scatter 100 points in a 2D space
n = 100
Yref = rand(2,n)

# Map into a 2D subspace of R^10
Q, R = qr(randn(10,2))
Q = Q[:,1:2]
X = Q*Yref

In [ ]:
# Compute pairwise distances
D = [norm(X[:,i]-X[:,j])^2 for i = 1:n, j = 1:n]

In [ ]:
function mds(D, k)
    n = size(D)[1]
    
    # Center and scale to get Gram matrix
    dmeans = [sum(D[:,j])/n for j=1:n]
    dd = sum(dmeans)/n
    B = [-0.5*(D[i,j]-dmeans[i]-dmeans[j]+dd) for i=1:n, j=1:n]
    B = (B+B')/2
    
    # Recover coords from eigenvectors
    λs, V = eigen(B)
    Y = (V[:,end-k+1:end] * diagm(sqrt.(λs[end-k+1:end])))'

    return Y, λs[end-k]
end

In [ ]:
Y, err = mds(D, 2)
println(err)

If we have done this right, we should have recovered "essentially" the same coordinates that we had before, in the sense that $QY - \bar{Y}^{\mathrm{ref}}$ is small for some $Q$, where $\bar{Y}^{\mathrm{ref}}$ is the centered version of the original data.  Finding the best possible orthogonal $Q$ to minimize the Frobenius norm error is an instance of an *orthogonal Procrustes* problem, a classic matrix nearness problem from numerical linear algebra that can be computed via an SVD.

In [ ]:
# Compute the centered reference system
barYref = Yref .- [sum(Yref[i,:])/n for i=1:2]

# Find the optimal rotation for matching
U, S, V = svd(barYref*Y')
Q = U*V'

# Report the mismatch
norm(barYref - Q*Y)

Usually, we will not be so fortunate as to have this type of reference coordinates, but the magnitude of the discarded eigenvalues gives us the norm of the mismatch to the spectral coordinates.

Often, MDS is used in a setting where we *only* can get pairwise distance information.  However, if we already have a centered data matrix $\bar{X} \in \mathbb{R}^{d \times n}$, we do not need to the squared distance matrix to find an effective coordinate system.  Instead, we can decompose $\bar{X} = U \Sigma V^T$, and use the mapping $x \mapsto U_k^T x$ to get the lower-dimensional embedding $Y = \Sigma_k V_k^T$ that optimally preserve distances.  That is, classical MDS and PCA are equivalent to each other; but classical MDS is typically used when we only have distances.

## Landmarks and Nystrom approximation

As we have seen before, we can typically construct a representation of a low-rank matrix with a subset of rows and columns of the matrix.  We have referred to this as the CUR decomposition; in the symmetric case, it is often known as the Nystrom approximation.  That is, if $A$ is a square symmetric matrix with rank $r$, then there is a symmetric permutation such that $A_{11} \in \mathbb{R}^{r \times r}$ is invertible and
$$
  A = \begin{bmatrix} A_{11} & A_{21}^T \\ A_{21} & A_{22} \end{bmatrix}
    = \begin{bmatrix} A_{11} \\ A_{21} \end{bmatrix} A_{11}^{-1} \begin{bmatrix} A_{11} & A_{21}^T \end{bmatrix}.
$$
If we take the economy QR decomposition
$$
  \begin{bmatrix} A_{11} \\ A_{21} \end{bmatrix} = QR
$$
we can then decompose $R A_{11}^{-1} R^T = U \Lambda U^T$ and get the (truncated) eigendecomposition
$$
  A = (QU) \Lambda (QU)^T
$$
What does this have to do with MDS?  Consider the case now where we use the Nystrom approximation of the squared distance matrix $D$:
$$
  D = \begin{bmatrix} D_{11} \\ D_{21} \end{bmatrix} D_{11}^{-1} \begin{bmatrix} D_{11} & D_{21}^T \end{bmatrix}.
$$
Centering gives
$$
  B = Z D_{11}^{-1} Z^T
$$
where $Z = P D_{:,1}$.  Taking the QR decomposition allows us to get the largest eigenvectors as before.

The selected columns in this scheme correspond to "landmarks."  If we choose $q$ landmarks, the time required for the method is $O(nq^2)$ rather than $O(n^3)$, and we only need to evaluate $nq$ distances (from each landmark to all other points).

This process also gives us an affine map that we can apply to embed new points in the lower-dimensional space when they arrive: that is

1.  Compute the vector $d \in \mathbb{R}^q$ of distances to each landmark, and let $z = d-\bar{d}$ be the "centered" version that comes from subtracting off the mean distances to the landmarks across the original training data.  We note that the vector $\bar{d}$ does *not* account for the new point being embedded.
2.  The $k$-dimensional embedding vector $y$ is now given by $y = \Lambda_k^{-1/2} R_k^{-T} z$ where $\Lambda_k$ and $R_k$ refer to the leading $k \times k$ submatrices in $\Lambda$ and $R$ above.

### Task (3 points)

Apply the landmark MDS computation described above to the test data set.

In [ ]:
# TODO: Implement the landmark method above using the first four points as landmarks

## Uncomment below to check that Y2 is correct (you should get close to zero error)

# # Find the optimal rotation for matching
# U, S, V = svd(barYref*Y2')
# Q = U*V'

# # Report the mismatch
# norm(barYref - Q*Y2)

## Manifold learning with Isomap

Now we consider a "Swiss roll" type of example that is poorly handled by methods like MDS.

In [ ]:
Y = rand(1000)
Z = rand(1000)
Xm = [ ((1 .+ Y) .* cos.(4*π*Y))' ;
       ((1 .+ Y) .* sin.(4*π*Y))' ;
       0.5*Z']
scatter(Xm[1,:], Xm[2,:], mz=Y, mc=:thermal)

In order to recover an "unrolled" coordinate system that describes this data, we want to find some sense of the nearby neighbors of each point.  We will keep track of all points that are within a radius of 0.3 of each other.  There are smarter algorithms than brute force search, but we will use the brute force approach for now. 

In [ ]:
# Construct epsilon-nearest-neighbors graph
filter(d) = if d < 0.35 d else 0.0 end
DG = sparse([filter(norm(Xm[:,i]-Xm[:,j])) for i = 1:1000, j = 1:1000])
G = MatrixNetwork(DG)

### Task (3 points)

Once we have the neighborhood graph, we will find all shortest path distances through the graph via Floyd-Warshall, then apply classical MDS to try to recover the geometry.

In [ ]:
# Floyd-Warshall computes all-pairs shortest paths in graph (D is the path distance)
D, P = floydwarshall(G)

# TODO: Apply classical MDS to D.^2 to compute Yiso and Ziso

In [ ]:
## Plot the Isomap recovered Y and Z
# scatter(Yiso, Ziso, mz=Y, mc=:thermal)

In [ ]:
# # Plot Y vs Yiso
# scatter(Y, Yiso)

In [ ]:
# # Plot Z vs Ziso (this shouldn't work as well)
# scatter(Z, Ziso)

### Task (3 points)

Use the Landmark version of MDS to implement Isomap more quickly.  The function

    d, pred = dijkstra(G, j)

can be used to compute the single-source shortest path vector $d = D_{:,j}$ (in $n \log n$ time).  Choose ten or twenty landmarks (columns of $D$).